In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()

driver.get("https://hatocase.com/tu-thiet-ke-op-lung-dien-thoai/")

element = driver.find_element(By.CSS_SELECTOR, ".row.row-collapse.row-full-width")
elements = element.find_elements(By.CLASS_NAME, "col-inner")
wait = WebDriverWait(driver, 30)
file = open("data.csv", "w", encoding="utf-8")

for element in elements:
    group = element.text
    element.click()
    wait.until(lambda x: x.find_element(By.CLASS_NAME, "mfp-wrap"))
    wrapper = driver.find_element(By.CLASS_NAME, "mfp-wrap")
    close_el = wrapper.find_element(By.CLASS_NAME, "mfp-close")
    content = wrapper.find_element(By.CLASS_NAME, "mfp-content")
 
    lightbox = content.find_element(By.CLASS_NAME, "lightbox-by-id")
    accordion_list = lightbox.find_elements(By.CLASS_NAME, "accordion")
    if len(accordion_list) > 0:
        accordion_item_list = accordion_list[0].find_elements(By.CLASS_NAME, "accordion-item")
        for accordion_item in accordion_item_list:
            accordion_item.click()
            accordion_title = accordion_item.find_element(By.CSS_SELECTOR, ".accordion-title span").text
            row = accordion_item.find_element(By.CLASS_NAME, "row")
            phones = row.find_elements(By.CLASS_NAME, "col")
            for phone in phones:
                a_tag = phone.find_element(By.TAG_NAME, "a")
                wait.until(EC.presence_of_element_located((By.TAG_NAME, "span")))
                href = a_tag.get_attribute("href")
                text = a_tag.find_element(By.TAG_NAME, "span").text
                
                if "#dongdienthoaikhac" not in href:
                    file.write(group + "," + accordion_title + "," + text + "," + href + "\n")
    else:
        row = content.find_element(By.CLASS_NAME, "row")
        phones = row.find_elements(By.CLASS_NAME, "col")
        for phone in phones:
            text = phone.find_element(By.TAG_NAME, "span").text
            href = phone.find_element(By.TAG_NAME, "a").get_attribute("href")
            if "#dongdienthoaikhac" not in href:
                file.write(group + ",None," + text + "," + href + "\n")
    close_el.click()


file.close()
driver.quit()    

In [2]:
import requests
from bs4 import BeautifulSoup    
import re
import json    
file_write = open("data_full.csv", "w", encoding="utf-8")
file = open("data.csv", "r", encoding="utf-8")
file_write.write("Group,SubGroup,Phone,Link,Image,Background,Upload,Border\n")
for line in file:
    group, sub_group, text, href = line.split(",")
    res = requests.get(href)
    soup = BeautifulSoup(res.text, 'html.parser')
    product = soup.select_one('.product-gallery')
    href = product.find('a').get_attribute_list('href')[0]
    Javascript = product.find('script')
    if Javascript:
        pattern = r'var fpdProductsJSON = (\[\[\{.*?\}\]\])'
        match = re.search(pattern, Javascript.string)
        json_text = match.group(1)
        json_parse = json.loads(json_text)
        elements = json_parse[0][0]["elements"]
        images = ""
        for element in elements:
            images = images + "," + element["source"]
        file_write.write(line.replace("\n", "") + "," + href + images + "\n")

file_write.close()
file.close()

In [ ]:
import re
import json
res = requests.get("https://hatocase.com/in-op-lung-dien-thoai-xiaomi-12t-pro-theo-yeu-cau/")
soup = BeautifulSoup(res.text, 'html.parser')
product = soup.select_one('.product-gallery')
href = product.find('a').get_attribute_list('href')[0]
Javascript = product.find('script').string
pattern = r'var fpdProductsJSON = (\[\[\{.*?\}\]\])'
match = re.search(pattern, Javascript)
json_text = match.group(1)
json_parse = json.loads(json_text)
elements = json_parse[0][0]["elements"]
for element in elements:
    print(element["source"])

In [4]:
count = 0
with open("data.csv", "r", encoding="utf-8") as f:
    for num, line in enumerate(f, 1):
        group, sub_group, text, href = line.split(",")
        if text == "":
            count += 1
            print(num, line)
print(count)

ValueError: too many values to unpack (expected 3)